<a href="https://colab.research.google.com/github/zhhhling/risk_limiting_dispatchSept/blob/main/118bus_proposed_neural_policy_for_rld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# in case gpu is used
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [3]:
import os, time
import cvxpy as cp
import random
from numpy import savetxt
import argparse
import json
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np

from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
from torch.autograd import Variable

root_path = './gdrive/MyDrive/Inbox/rld_neural_policy/'

# helper functions, can be folded

## utils.py

In [4]:
def create_dir(PATH):
    isExist = os.path.exists(PATH)

    if not isExist:
        # Create a new directory if it does not exist 
        os.makedirs(PATH)
        print("The new directory is created!")


# Evaluate using vector distance
def measure_relative_distance(v1, v2):
    '''
        Note that v1 is the benchmark.
        Norm is calculated along dimension/axis 1
        and average is calculated along dimension/axis 0
        Also return the distance vetor.
    '''
    if len(v1.shape)==1 and len(v2.shape)==1:
        distance = np.abs(v1-v2)/np.abs(v1)

    if len(v1.shape)==2 and len(v2.shape)==2:
        distance = np.linalg.norm(v1-v2, axis=1)/np.linalg.norm(v1, axis=1)

    return distance, np.mean(distance)

## plot.py

In [5]:
def plot_train_loss(train_hist, epoch, fig_name, yplot):
        print('len:', len(train_hist['train_losses']))

        fig = plt.figure(figsize=(10,10))
        plt.ticklabel_format(style='plain', scilimits=(-3, 3), useOffset=False)
        ax = fig.add_subplot(1, 1, 1)
        if yplot == 'log10':
            ax.plot(range(len(train_hist['train_losses'])), np.log10(train_hist['train_losses']), label='train loss')
            ax.set_ylabel('log10(train loss)',fontsize=20)
        elif yplot == 'plain':
            ax.plot(range(len(train_hist['train_losses'])), train_hist['train_losses'], label='train loss')
            ax.set_ylabel('train loss',fontsize=20)
        ax.set_xlabel('training epochs',fontsize=20)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        plt.savefig(saved_path+str(epoch)+fig_name)

## models.py

In [6]:
class ActionNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(ActionNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, H)
        self.linear3 = torch.nn.Linear(H, H)
        self.linear4 = torch.nn.Linear(H, H)
        self.linear5 = torch.nn.Linear(H, D_out)

        # Define proportion or neurons to dropout
        self.dropout = nn.Dropout(0.25)


    def forward(self, input):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """

        x = F.relu(self.linear1(input))
        x = self.dropout(x)
        x = F.relu(self.linear2(x))
        x = self.dropout(x)
        x = F.relu(self.linear3(x))
        x = self.dropout(x)
        x = F.relu(self.linear4(x))
        x = self.dropout(x)
        y_pred = F.relu(self.linear5(x))


        return y_pred


class RewardNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(RewardNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, H)
        self.linear3 = torch.nn.Linear(H, H)
        self.linear4 = torch.nn.Linear(H, H)
        self.linear5 = torch.nn.Linear(H, D_out)

    def forward(self, input):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
       
        x = F.relu(self.linear1(input))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        y_pred = torch.tanh(self.linear5(x))


        return y_pred


class TrainDataset(Dataset):
    def __init__(self, samples):
        self.samples = samples
    # must override two of the subclass functions:
    def __len__(self):
        # return len(self.samples)
        return self.samples.shape[0]

    def __getitem__(self, idx):
        # return self.samples[idx]
        return self.samples[idx,:]


def init_weights(m):
    if isinstance(m, nn.Linear):

        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        m.bias.data.fill_(0.001)
        # m.bias.data.uniform_(0, y)

        # torch.nn.init.xavier_uniform_(m.weight)
        # m.bias.data.fill_(0.01)

        # How to call this function:
        # net.apply(init_weights)


## gauge.py

In [7]:
def gauge_function(V, G, H):
    """
    The gauge function of the vector z w.r.t. the set P
    is given by the following code.

    V can be batched, for example, r-dimensional and batch size of K, then shape(V) = (r, K)
    Note that the second dimension is the batch.

    P is defined by {v: G@v <= Hj} = {v: g_i^T@v <= Hj_i, i = 1, ..., q}
    shape(G) = (q, r), shape(Hj) = (q, 1), shape(H) = (q, K)
    P must contain the origin in its interior.
    
    """
    # return torch.max(G@V/H,dim = 0).values # shape(output) = (1, K)

    # torch.div() for element-wide division
    return torch.max(torch.div(G@V, H),dim = 0).values # shape(output) = (1, K)

def gauge_map(Z, G, H):
    """
    For any Z \belongsto B_infinity, the gauge map from B_infinity to the set P
    defined by {v: G@v <= h} is given by the following code.
    
    Z can be batched, for example, r-dimensional and batch size of K, then shape(V) = (r, K)
    Note that the second dimension is the batch.

    P is defined by {v: G@v <= Hj} = {v: g_i^T@v <= Hj_i, i = 1, ..., q}
    shape(G) = (q, r), shape(Hj) = (q, 1), shape(H) = (q, K)
    P must contain the origin in its interior.
    """

    gamma_dest = gauge_function(Z, G, H)# shape = (1, K)
    # print('gamma_dest:', gamma_dest)
    gamma_start = torch.linalg.norm(Z, ord = np.inf, dim=0) # shape(1, K)

    scaling_mat = torch.diag(gamma_start/gamma_dest) # shape = (K, K)

    return Z@scaling_mat # shape = (r, K), this is the new point in P

## system.py

In [8]:
def identify_unique_lines(connections):
    all_lines = {}
    count = 0
    for line in connections:
        all_lines[count] = line
        count+=1

    # This code snippet only finds out the repeated lines with exactly the same order of nodes,
    # but not deal with that [i,j] and [j,i] are also repeated lines
    # By checking connections, there is no repeated lines like [i,j] and [j,i]
    unique_lines = {}
    for k, val in all_lines.items():
        if val not in unique_lines.values():
            unique_lines[k]=val
    print('unique_lines length:', len(unique_lines))

    repeated_lines = [[42, 49],[49, 54],[56, 59],[49, 66],[77, 80],[89, 90],[89, 92]]
    # For example, [42, 49] appears twice
    set1 = {}
    set2 = {}
    for k, val in all_lines.items():
        if val in repeated_lines and k in unique_lines:
            set1[val[0], val[1]] = k # Record the repeated lines when they first appear
        if val in repeated_lines and k not in unique_lines:
            set2[val[0], val[1]] = k # Record the repeated lines when they appear more than once

    # print('set1:', len(set1))
    # print('set2:', len(set2))

    return unique_lines, set1, set2


def get_Y(N, B):
    # N = num_buses
    Y = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if i==j : 
                Y[i,j] = sum(B[i,:])
            else: 
                Y[i,j] = -B[i,j]

    return Y[:,1:]


def get_A(N, L, B, connections):
    # N = num_buses
    # L = num_lines 
    A = np.zeros((L, N))

    for i, line in enumerate(connections):
        row = line[0]-1
        col = line[1]-1
        A[i, row] = B[row,col]
        A[i, col] = -B[row,col]

    return A[:,1:]


# Import 118bus
def import_118bus(params_path):
    bus_data_fname = '118bus_BusData.csv'
    gen_data_fname = '118bus_GenData.csv'

    branch_data_fname = '118bus_BranchData.csv'
    cost_data_fname = '118bus_CostData.csv'

    bus_data_df = pd.read_csv(params_path+bus_data_fname,header=None)
    gen_data_df = pd.read_csv(params_path+gen_data_fname,header=None)
    branch_data_df = pd.read_csv(params_path+branch_data_fname,header=None)
    cost_data_df = pd.read_csv(params_path+cost_data_fname,header=None)


    num_buses = bus_data_df.shape[0]
    num_lines = branch_data_df.shape[0]
    num_gens = gen_data_df.shape[0]


    bus_data = bus_data_df.to_numpy()
    gen_data = gen_data_df.to_numpy()
    branch_data = branch_data_df.to_numpy()
    cost_data = cost_data_df.to_numpy()

    x = branch_data[:,3]
    print('max of x:', max(x))
    print('min of x:', min(x))


    b = 1/x
    Z0 = 10
    b = b/Z0
    print('max of b:', max(b))
    print('min of b:', min(b))

    connections = []
    branches = branch_data[:,:2]
    for i in range(branches.shape[0]):
            connections.append([int(branches[i,0]),int(branches[i,1])])
    print('len of connections:', len(connections))

    unique_lines, set1, set2 = identify_unique_lines(connections)

    B = np.zeros((num_buses, num_buses))
    for k, line in unique_lines.items():
        row = line[0]-1
        col = line[1]-1
        B[row, col] = b[k]
        B[col, row] = b[k]

    PD = bus_data[:,2]/20
    print('Total PD:', sum(PD))

    return num_buses, num_lines, B, connections, PD

## dataloader.py

In [9]:

def generate_inputs(Ntr, Ntst, Ntr2, scaling, nominal_PD_data, data_path):


    forecasts = nominal_PD_data.reshape(1,-1)

    lb = np.clip((1-scaling)*forecasts, a_min=0., a_max=None)
    ub = (1+scaling)*forecasts

    suffix = str(int(scaling*100))+'percent.npy'
    print('suffix:', suffix)


    train_dataset = np.random.uniform(lb, ub, (Ntr, N))
    test_dataset = np.random.uniform(lb, ub, (Ntst, N))

    # Generate dataset for pretrain
    random_rows = np.random.choice(Ntr, Ntr2)
    pretrain_dataset = train_dataset[random_rows,:]
    print('pretrain_dataset:', pretrain_dataset.shape)

    np.save(data_path+'train_set.npy', train_dataset)
    np.save(data_path+'test_set.npy', test_dataset)
    np.save(data_path+'pretrain_set.npy', pretrain_dataset)

    return train_dataset, test_dataset, pretrain_dataset


## benchmarks.py

In [10]:
def saa_solver(forecast, num_sce):
    '''
    Given a single instance of forecast, solve the SAA version
    of the true problem.
    This is called cp policy in our work.
    '''

    M = num_sce

    x = cp.Variable((N, 1))
    theta = cp.Variable((N-1,M))
    y = cp.Variable((N, M))

    constraints_list = []

    for m in range(M):

        if Pe == 'uniform':
            lb = np.clip((1-ratio)*forecast, a_min=0., a_max=None)
            ub = (1+ratio)*forecast
            rt_scenario = np.random.uniform(lb, ub, (forecast.shape[0], forecast.shape[1]))

        else:

            rt_scenario = forecast + sigma*np.random.randn(N, 1) 
        
        d_tilde = rt_scenario - x

        constraints_list.append( y[:,m:m+1] == Yrr@theta[:,m:m+1] + d_tilde )
        
    constraints_list.append( Arr@theta <= Fmax*np.ones((num_lines, M)))
    constraints_list.append( Arr@theta >= -Fmax*np.ones((num_lines, M)))

    constraints_list.append( theta<=np.pi)
    constraints_list.append( theta>=-np.pi)
    constraints_list.append( x>=0)

    Q_vals = []
    for m in range(M):

        Q_vals.append(cp.pos(y[:,m:m+1].T)@linear_cost_Coeff)

    Q_pred = cp.sum(Q_vals)/M
    prob = cp.Problem(cp.Minimize( da_cost_Coeff.T@x + Q_pred ), constraints_list)
   
    prob.solve(verbose = False, solver = cp.ECOS) ## Works!

    return x.value.T, prob.value


def ap_solver(forecast, num_sce):
    '''
    Given a single instance of forecast, solve the SAA version
    of the true problem, but with theta approximated by affine policy.
    This is called ap policy in our work.
    '''

    M = num_sce

    c = da_cost_Coeff
    q = linear_cost_Coeff
    B = -Yrr
    F = Arr
    rho_lb = 10.
    rho_ub = 10.

    
    Pi = cp.Variable((N-1,N))
    beta = cp.Variable((N-1,1))

    x = cp.Variable((N, 1))


    constraints_list = []

    Q_vals = []

    for m in range(M):
        # for each scenario of noise
        if Pe == 'uniform':
            lb = np.clip((1-ratio)*forecast, a_min=0., a_max=None)
            ub = (1+ratio)*forecast
            rt_scenario = np.random.uniform(lb, ub, (forecast.shape[0], forecast.shape[1]))

        else:

            rt_scenario = forecast + sigma*np.random.randn(N, 1) 

        noise_sce =  rt_scenario - forecast
        net_d = rt_scenario - x

        theta_pred = Pi@noise_sce + beta
        y_pred = net_d - B@theta_pred

        p_f_lb = rho_lb*cp.norm( cp.pos(-Fmax*np.ones((num_lines, 1)) - F@theta_pred) )
        p_f_ub = rho_ub*cp.norm( cp.pos(F@theta_pred -Fmax*np.ones((num_lines, 1))) )

        p_v_lb  = rho_lb*cp.norm( cp.pos(-np.pi*np.ones((N-1, 1)) - theta_pred) )
        p_v_ub  = rho_ub*cp.norm( cp.pos(theta_pred - np.pi*np.ones((N-1, 1))) )

        Q_val = q.T@cp.pos(y_pred) + p_f_lb + p_f_ub + p_v_lb + p_v_ub

        Q_vals.append(Q_val)

    constraints_list.append( x>=0)


    Q_pred = cp.sum(Q_vals)/M

    prob = cp.Problem(cp.Minimize( c.T@x + Q_pred ), constraints_list)

    prob.solve(verbose = False, solver = cp.ECOS) ## Works!  

    assert prob.status not in ["infeasible", "unbounded"], "Problem is not feasible or unbounded."


    return x.value.T, prob.value


def solver_outer_loop(forecasts, num_sce, solver_name):
    '''
    Given a batch of instances and solve rld problem for each instance.
    We only care about x for now.
    Can return computation time for each instance at the same time (in minutes)
    '''

    num_points = forecasts.shape[0]

    x_vals = []
    total_cost_vals = []
    times = []


    for it in range(num_points):

        forecast = forecasts[it,:].reshape(-1,1)

        start_time = time.time()

        if solver_name == 'saa':
            x_value, prob_value = saa_solver(forecast, num_sce)

        elif solver_name == 'ap':
            x_value, prob_value = ap_solver(forecast, num_sce)

        else:
            print('No such solver exists!')

        end_time = time.time()
        times.append((end_time-start_time)/60)

        x_vals.append(x_value.flatten())
        total_cost_vals.append(prob_value)

        # print("--- %s minutes ---" % ((time.time() - start_time)/60))
        
    x_vals = np.array(x_vals)
    total_cost_vals = np.array(total_cost_vals).reshape(-1,1)
    times = np.array(times)

    return x_vals, total_cost_vals, times


def evaluate_Q(forecast, x, num_sce):
    '''
    Given a single instance and the initial dispatch x, and evaluate the quality
    of x.
    This function can be applied to the decision obtained from any policy.
    '''

    M = num_sce

    theta = cp.Variable((N-1,M))
    y = cp.Variable((N, M))

    constraints_list = []

    for m in range(M):
        if Pe == 'uniform':
            lb = np.clip((1-ratio)*forecast, a_min=0., a_max=None)
            ub = (1+ratio)*forecast
            rt_scenario = np.random.uniform(lb, ub, (forecast.shape[0], forecast.shape[1]))

        else:

            rt_scenario = forecast + sigma*np.random.randn(N, 1) 
        
        net_d = rt_scenario - x

        constraints_list.append( y[:,m:m+1] == Yrr@theta[:,m:m+1] + net_d )
        
    constraints_list.append( Arr@theta <= Fmax*np.ones((num_lines, M)))
    constraints_list.append( Arr@theta >= -Fmax*np.ones((num_lines, M)))

    constraints_list.append( theta<=np.pi)
    constraints_list.append( theta>=-np.pi)

    Q_vals = []
    for m in range(M):

        Q_vals.append(cp.pos(y[:,m:m+1].T)@linear_cost_Coeff)

    Q_pred = cp.sum(Q_vals)/M

    prob = cp.Problem(cp.Minimize( Q_pred ), constraints_list)

    prob.solve(verbose = False, solver = cp.ECOS) ## Works!   

    return prob.value, np.sum(y.value, axis=1)/M


def evaluate_outer_loop(forecasts, x_pred, num_sce):
    '''
    Given a batch of instances and the associated predictions of first stage decision,
    and evaluate these initial dispatch decisions for each instance.

    '''

    num_points = forecasts.shape[0]

    Q_pred = []
    y_pred = []

    for i in range(num_points):
        forecast = forecasts[i,:].reshape(-1,1)
        init_dispatch = x_pred[i,:].reshape(-1,1)

        Q_value, avg_y_value = evaluate_Q(forecast, init_dispatch, num_sce)

        Q_pred.append(Q_value)
        y_pred.append(avg_y_value.flatten())
        

    Q_pred = np.array(Q_pred).reshape(-1,1)
    y_pred = np.array(y_pred)
    

    total_cost_pred = x_pred@da_cost_Coeff + Q_pred

    return total_cost_pred, Q_pred, y_pred


# main.py

## Define params_path, saved_path, data_path, and model_path here




In [11]:
def get_paths():
    params_path = root_path+'118bus/params/'
    data_path = root_path+'118bus/rld_simulations/data/'
    saved_path = root_path+'118bus/simulations/results/'
    model_path = root_path+'118bus/rld_simulations/saved_models/'


    isExist = os.path.exists(data_path)
    if not isExist:
        # Create a new directory if it does not exist 
        os.makedirs(data_path)

    
    isExist = os.path.exists(saved_path)
    if not isExist:
        # Create a new directory if it does not exist 
        os.makedirs(saved_path)

    return params_path, data_path, saved_path, model_path

## Import 118bus system

In [12]:
def load_saved_models(model_path):

    action_net = ActionNet(N, 256, N)
    action_net.load_state_dict(torch.load(model_path+'trained_action_net.pt'))

    reward_net = RewardNet(N, 256, N-1)
    reward_net.load_state_dict(torch.load(model_path+'trained_reward_net.pt'))

    return action_net, reward_net


def load_pretrain_models(model_path):

    action_net = ActionNet(N, 256, N)
    action_net.load_state_dict(torch.load(model_path+'trained_action_net_sl.pt'))

    reward_net = RewardNet(N, 256, N-1)
    reward_net.load_state_dict(torch.load(model_path+'trained_reward_net_sl.pt'))

    return action_net, reward_net


def load_cost_coeff(params_path):    
    quad_cost_coeff = np.load(params_path+'quad_cost_coeff.npy')
    linear_cost_coeff = np.load(params_path+'linear_cost_coeff.npy')
    da_cost_coeff = np.load(params_path+'da_cost_coeff.npy')

    return quad_cost_coeff, linear_cost_coeff, da_cost_coeff


params_path, data_path, saved_path, model_path = get_paths()


num_buses, num_lines, B, connections, PD_data = import_118bus(params_path)

N = num_buses
L = num_lines 

Yrr = get_Y(num_buses, B)
Arr = get_A(num_buses, num_lines, B, connections)

# Define feasibility set for gauge mapping
G = np.block([
              [Arr],
              [-Arr],
              [np.eye(N-1)],
              [-np.eye(N-1)]
])
print('G shape:', G.shape)


Fmax = 1.


# If forecast error follows gaussian distribution with zero mean and std sigma
Pe = 'gaussian'
sigma = 1.

# If forecast error follows interval bounded distribution-uniform then ratio is 5%
# Pe = 'uniform'
ratio = 0.05

# Load cost coefficients
quad_cost_coeff, linear_cost_coeff, da_cost_coeff = load_cost_coeff(params_path) 

quad_cost_Coeff = np.diag(quad_cost_coeff)
linear_cost_Coeff = linear_cost_coeff.reshape(-1, 1)
da_cost_Coeff = da_cost_coeff.reshape(-1,1)

print('quad_cost_Coeff shape:', quad_cost_Coeff.shape)
print('linear_cost_Coeff shape:', linear_cost_Coeff.shape)
print('da_cost_Coeff shape:', da_cost_Coeff.shape)

max of x: 0.411
min of x: 0.004
max of b: 25.0
min of b: 0.24330900243309003
len of connections: 186
unique_lines length: 179
Total PD: 212.10000000000005
G shape: (606, 117)
quad_cost_Coeff shape: (118, 118)
linear_cost_Coeff shape: (118, 1)
da_cost_Coeff shape: (118, 1)


# load.py

In [13]:
nominal_PD_data = PD_data.reshape(1,-1)

# Load dataset
scaling = 0.1
Ntr = 50000
Ntr2 = 100
Ntst = 100

# _, test_dataset, pretrain_dataset =generate_inputs(Ntr, Ntst, Ntr2,
#                                                     scaling, nominal_PD_data, 
#                                                     data_path)

train_dataset = np.load(data_path+'train_set.npy')
test_dataset = np.load(data_path+'test_set.npy')
pretrain_dataset = np.load(data_path+'pretrain_set.npy')

max_forecast_val = np.max(train_dataset)
# Note that the original train_dataset and test_dataset are not normalized
TrainSet = TrainDataset(train_dataset/max_forecast_val)
TestSet = TrainDataset(test_dataset/max_forecast_val)


# pretrain.py

In [14]:
# Format:
# X_train, X_test: [forecasts, x_true]
# X_train2, X_test2:[forecasts-x_true, theta_true, y_true, Q_true]

X_train = np.load(data_path+'X_train.npy')
X_test = np.load(data_path+'X_test.npy')
X_train2 = np.load(data_path+'X_train2.npy')
X_test2 = np.load(data_path+'X_test2.npy')

## pretrain action net

In [26]:
def train_action(dataset, num_epochs, batch_size, yplot = 'plain'):

    # Set learning rates
    lr = 1e-5

    # Build models or load models
    # action_net, reward_net = load_saved_models()
    action_net = ActionNet(N, 256, N)
    action_net.apply(init_weights)
    
    # action_net.load_state_dict(torch.load(saved_path+'trained_action_net_sl.pt'))

    # return action_net

    # Configure optimizers
    # action_optim = optim.Adam(action_net.parameters(), lr=lr, weight_decay=1e-2)
    action_optim = optim.Adam(action_net.parameters(), lr=lr, betas=(0.5, 0.999))

    loss_criterion = nn.MSELoss()

    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    train_hist = {}
    train_hist['train_losses'] = []
    train_hist['per_epoch_time'] = []
    train_hist['total_time'] = []
    num_iter = 0

    print('Training starts!')
    start_time = time.time()

    for epoch in range(num_epochs):
        train_losses = []
        epoch_start_time = time.time()

        for batch in train_loader:

            ## zero grad
            action_net.zero_grad()

            ## get a batch of data
            input = batch.float()
            load = input[:, :N]
            target = input[:, N:]

            x_pred = action_net(load)

            train_loss = loss_criterion(x_pred, target)

            train_loss.backward(retain_graph=True)
            action_optim.step()

            train_losses.append(train_loss.item())


            num_iter += 1

            if num_iter % 2000 == 0:
                print('[%d/%d], loss: %.5f' % ((epoch + 1), 
                    (num_iter + 1), 
                    torch.FloatTensor(train_losses).mean().item()))
                    
        epoch_end_time = time.time()
        per_epoch_time = epoch_end_time - epoch_start_time

        loss_value = torch.mean(torch.FloatTensor(train_losses)).item()

        train_hist['train_losses'].append(round(loss_value, 4))
        train_hist['per_epoch_time'].append(per_epoch_time/60)
                
        if epoch > 0 :
            if epoch % 20000 == 0 or epoch == num_epochs-1:

                torch.save(action_net.state_dict(), saved_path+'pretrain_action_net.pt')

                plot_train_loss(train_hist, epoch, 'pretrain_action_loss.png', yplot)
                
    end_time = time.time()
    total_time = end_time - start_time
    train_hist['total_time'].append(total_time)

    print("Avg per epoch time: %.2f minutes, total %d epochs time: %.2f minutes" % \
        (torch.mean(torch.FloatTensor(train_hist['per_epoch_time'])), num_epochs, total_time/60))
    print("Training finish!")

    return action_net


## pretrain reward net

In [27]:
def train_reward(dataset, num_epochs, batch_size, yplot = 'plain'):

    # Set learning rates
    lr = 0.0001

    # Build models or load models
    # action_net, reward_net = load_saved_models()
    reward_net = RewardNet(N, 256, N-1)

    # reward_net.load_state_dict(torch.load(saved_path+'trained_reward_net_sl.pt'))

    # return reward_net
    

    # Configure optimizers
    reward_optim = optim.Adam(reward_net.parameters(), lr=lr, betas=(0.5, 0.999))

    loss_criterion = nn.MSELoss()

    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    train_hist = {}
    train_hist['train_losses'] = []
    train_hist['per_epoch_time'] = []
    train_hist['total_time'] = []
    num_iter = 0

    G_ts = torch.tensor(G).float()
    B_ts = torch.tensor(-Yrr).float()
    F_ts = torch.tensor(Arr).float()
    q_ts = torch.tensor(linear_cost_Coeff).float()
    c_ts = torch.tensor(da_cost_Coeff).float()

    print('Training starts!')
    start_time = time.time()

    for epoch in range(num_epochs):
        train_losses = []
        epoch_start_time = time.time()

        for batch in train_loader:

            ## zero grad
            reward_net.zero_grad()

            ## get a batch of data
            input = batch.float()
            q_input = input[:, :N]
            target = input[:, N:N+N-1]
            y_target = input[:, N+N-1:N+N-1+N]
            Q_target = input[:, N+N-1+N:]

            pred = reward_net(q_input)

            # net_d = q_input*max_val
            net_d = q_input

            h_batch = torch.cat([
                                Fmax*torch.ones(L, input.shape[0]),
                                Fmax*torch.ones(L, input.shape[0]),
                                np.pi*torch.ones(N-1, input.shape[0]),
                                np.pi*torch.ones(N-1, input.shape[0])
                                ],dim=0)

            h_batch_ts = torch.tensor(h_batch).float()
            
            ctr_ts = torch.zeros((N-1, input.shape[0]))
            assert torch.max(G_ts@ctr_ts - h_batch_ts)< 0.0, "Caution: v0 is not a feasible point."

            # Use ctr_ts to shift the feasible set Gv<=H
            h_shift = h_batch_ts - G_ts@ctr_ts
            assert torch.min(h_shift) > 0.0, "Caution: origin is not an interior point"

            # Use gauge mapping to transform output to feas_output

            feas_pred = gauge_map(pred.T, G_ts, h_shift) + ctr_ts

            y_pred = -B_ts@feas_pred + net_d.T # (N, batch_size)
            Q_pred = F.relu(y_pred.T)@q_ts

            train_loss = loss_criterion(feas_pred.T, target)

            train_loss.backward(retain_graph=True)
            reward_optim.step()

            train_losses.append(train_loss.item())

            num_iter += 1

            if num_iter % 10000 == 0:
                print('[%d/%d], loss: %.5f' % ((epoch + 1), 
                    (num_iter + 1), 
                    torch.FloatTensor(train_losses).mean().item()))
                    
        epoch_end_time = time.time()
        per_epoch_time = epoch_end_time - epoch_start_time

        loss_value = torch.mean(torch.FloatTensor(train_losses)).item()

        train_hist['train_losses'].append(round(loss_value, 4))
        train_hist['per_epoch_time'].append(per_epoch_time/60)
                
        if epoch > 0 :
            if epoch % 100000 == 0 or epoch == num_epochs-1:

                torch.save(reward_net.state_dict(), saved_path+'pretrain_reward_net.pt')

                plot_train_loss(train_hist, epoch, 'pretrain_reward_loss.png', yplot)
                
    end_time = time.time()
    total_time = end_time - start_time
    train_hist['total_time'].append(total_time)

    print("Avg per epoch time: %.2f minutes, total %d epochs time: %.2f minutes" % \
        (torch.mean(torch.FloatTensor(train_hist['per_epoch_time'])), num_epochs, total_time/60))
    print("Training finish!")

    return reward_net


# train.py

In [28]:
def train(dataset, num_epochs, num_sce, batch_size, yplot = 'plain', use_pretrain='True', use_saved=False):

    # Set learning rates
    lr = 0.0001

    # Build models or load models
    if use_pretrain and not use_saved:
        action_net, reward_net = load_pretrain_models(model_path)

        action_net.train(True)
        reward_net.train(True)

    elif not use_pretrain and use_saved:
        action_net, reward_net = load_saved_models(model_path)

        action_net.train(True)
        reward_net.train(True)

    else:
        action_net = ActionNet(N, 256, N)
        reward_net = RewardNet(N, 256, N-1)

    # Configure optimizers
    action_optim = optim.Adam(action_net.parameters(), lr=lr, betas=(0.5, 0.999))
    reward_optim = optim.Adam(reward_net.parameters(), lr=lr, betas=(0.5, 0.999))


    # Constant Parameters
    G_ts = torch.tensor(G).float()
    B_ts = torch.tensor(-Yrr).float()
    q_ts = torch.tensor(linear_cost_Coeff).float()
    c_ts = torch.tensor(da_cost_Coeff).float()
    max_forecast_ts = torch.tensor(max_forecast_val).float()

    h_batch = torch.cat([
                        Fmax*torch.ones(L, batch_size),
                        Fmax*torch.ones(L, batch_size),
                        np.pi*torch.ones(N-1, batch_size),
                        np.pi*torch.ones(N-1, batch_size)
                        ],dim=0)

    h_batch_ts = torch.tensor(h_batch).float()

    ctr_ts = torch.zeros((num_buses-1, batch_size))
    assert torch.max(G_ts@ctr_ts - h_batch_ts)< 0.0, "Caution: v0 is not a feasible point."

    # Use ctr_ts to shift the feasible set Gv<=H
    h_shift = h_batch_ts - G_ts@ctr_ts
    assert torch.min(h_shift) > 0.0, "Caution: origin is not an interior point"

    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    train_hist = {}
    train_hist['train_losses'] = []
    train_hist['per_epoch_time'] = []
    train_hist['total_time'] = []
    num_iter = 0

    print('Training starts!')
    start_time = time.time()

    for epoch in range(num_epochs):
        train_losses = []
        epoch_start_time = time.time()

        for batch in train_loader:

             ## zero grad
            action_net.zero_grad()
            reward_net.zero_grad()

            ## get a batch of data
            input = batch.float()
            x = action_net(input)# this input has been scaled, so x is treated as scaled

            Q_pred = 0.
            for m in range(num_sce):          
                if Pe == 'uniform':  
                    input_val = input.detach().numpy()
                    lb = np.clip((1-ratio)*input_val, a_min=0., a_max=None)
                    ub = (1+ratio)*input_val
                    rt_scenario = np.random.uniform(lb, ub, (input_val.shape[0], input_val.shape[1])) 
                    rt_scenario = torch.tensor(rt_scenario).float()*max_forecast_ts

                else:
                    noise_sce = sigma*torch.randn_like(input)
                    rt_scenario = input*max_forecast_ts + noise_sce

                q_input = rt_scenario - x*max_forecast_ts

                pred = reward_net(q_input)

                net_d = q_input # (N, bsize)

                # Use gauge mapping to transform output to feas_output
                feas_pred = gauge_map(pred.T, G_ts, h_shift) + ctr_ts

                # Construct cost function
                y_pred = -B_ts@feas_pred + net_d.T # (N, batch_size)
                # quad_cost = torch.diagonal(y.T@C1_ts@y)
                linear_cost = F.relu(y_pred.T)@q_ts
                # rt_cost += quad_cost + linear_cost.flatten()
                Q_pred += linear_cost.flatten()

            Q_pred = Q_pred/num_sce

            x_pred = x*max_forecast_ts

            da_cost = x_pred@c_ts
            total_cost = Q_pred + da_cost.flatten()

            train_loss = torch.mean(total_cost) 

            train_loss.backward(retain_graph=True)
            action_optim.step()
            reward_optim.step()

            train_losses.append(train_loss.item())


            num_iter += 1

            if num_iter % 5000 == 0:
                print('[%d/%d], loss: %.5f' % ((epoch + 1), 
                    (num_iter + 1), 
                    torch.FloatTensor(train_losses).mean().item()))
                
        epoch_end_time = time.time()
        per_epoch_time = epoch_end_time - epoch_start_time

        loss_value = torch.mean(torch.FloatTensor(train_losses)).item()

        train_hist['train_losses'].append(round(loss_value, 4))
        train_hist['per_epoch_time'].append(per_epoch_time/60)
                
        if epoch > 0 :
            if epoch % 200 == 0 or epoch == num_epochs-1:

                torch.save(action_net.state_dict(), saved_path+'trained_action_net.pt')

                torch.save(reward_net.state_dict(), saved_path+'trained_reward_net.pt')

                plot_train_loss(train_hist, epoch, 'train_loss.png', yplot)
                
    end_time = time.time()
    total_time = end_time - start_time
    train_hist['total_time'].append(total_time)

    print("Avg per epoch time: %.2f minutes, total %d epochs time: %.2f minutes" % \
        (torch.mean(torch.FloatTensor(train_hist['per_epoch_time'])), num_epochs, total_time/60))
    print("Training finish!")


    return action_net, reward_net


# call.py

In [18]:
# pretrain_action_net = train_action(TrainDataset(X_train), num_epochs=10, 
#                                    batch_size=1, yplot = 'log10')

In [19]:
# pretrain_reward_net = train_action(TrainDataset(X_train2), num_epochs=10, 
#                                    batch_size=1, yplot = 'log10')

In [ ]:
trained_action_net, trained_reward_net = train(TrainSet, num_epochs=1400, 
                                               num_sce=10, batch_size=100, 
                                               yplot = 'plain')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Training starts!


# eval.py


In [65]:
N_ap = 10
N_cp = 500
N_eval = 500

## Proposed neural policy

In [66]:
# Load neural solutions
# x_nu = np.load(data_path+'x_nu.npy')

# otherwise
# Load saved models from model_path or saved_path
action_net, _ = load_saved_models(model_path)

action_net.eval()

with torch.no_grad():

    input = torch.tensor(test_dataset/max_forecast_val).float()
    start_time = time.time()

    output = action_net(input)

    print("--- %s minutes ---" % ((time.time() - start_time)/60))

    output_val = output.detach().numpy()

    x_nu = output_val*max_forecast_val

--- 2.9969215393066407e-05 minutes ---


In [67]:
# Evaluate using evaluate_outer_loop(forecasts, x_pred)
total_cost_nu, Q_nu, y_nu = evaluate_outer_loop(test_dataset, x_nu, num_sce=N_eval)

np.save(saved_path+'x_nu.npy', x_nu)
np.save(saved_path+'total_cost_nu.npy', total_cost_nu)
np.save(saved_path+'Q_nu.npy', Q_nu)
np.save(saved_path+'y_nu.npy', y_nu)

# plotting